In [2]:
import platform
from qlib.contrib.data.handler import check_transform_proc
import qlib
from pathlib import Path
from qlib import data, model
from qlib.constant import REG_CN  # 'cn'
from qlib.data import D
from qlib.data.filter import ExpressionDFilter,NameDFilter
from qlib.utils import init_instance_by_config
from qlib.config import C

path = "F:\qlib_bin" if platform.node() == 'dimi-pc' else "D:\qlib_bin"
qlib.init(provider_uri={'day': Path(path)}, region=REG_CN)
# calendar max and min
D.calendar(freq='day')[[-1,0]]

[29048:MainThread](2023-08-04 09:42:17,704) INFO - qlib.Initialization - [config.py:415] - default_conf: client.
[29048:MainThread](2023-08-04 09:42:18,289) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[29048:MainThread](2023-08-04 09:42:18,291) INFO - qlib.Initialization - [__init__.py:76] - data_path={'day': WindowsPath('D:/qlib_bin')}


array([Timestamp('2023-06-02 00:00:00'), Timestamp('2005-01-04 00:00:00')],
      dtype=object)

In [15]:
# 只看沪深两市
# sh = NameDFilter(name_rule_re = 'SH',fstart_time='2020-01-01', fend_time='2023-07-14')
# sz = NameDFilter(name_rule_re = 'SZ',fstart_time='2020-01-01', fend_time='2023-07-14')
# tt = NameDFilter(name_rule_re = 'sz000001|sz002230|sz002602',fstart_time='2023-01-01', fend_time='2023-06-30')


# 设置股票池
# all_shsz = D.instruments(market='csi300', filter_pipe=[tt])
tt = D.instruments(market='test')
D.list_instruments(tt).keys()


dict_keys(['SZ000001', 'SZ002230', 'SZ002602'])

In [16]:
# dataLoader to get data
"""
example_loader_config = {
    'feature': (['expr_1', 'expr_2', ... , 'expr_n'],
               ['name_1', 'name_2', ... , 'name_n']), # name list是经过_parse_fields_info()函数处理过才有的
    ...,
    'label':  (['expr_1', 'expr_2', ... , 'expr_n'],
               ['name_1', 'name_2', ... , 'name_n']),    
}
"""
start_time = "2023-01-01"
end_time = "2023-06-30"
fit_start_time = start_time
fit_end_time = "2023-01-31"

"""
factor15
rank(correlation(rank(high), rank(volume), 3)：分别对最高价成交价和成交量进行计算排序分位值，然后计算二者近3天的相关系数并取排名分位值，本质上就是分析近3天最高价和成交量的相关性。
-1 * sum(rank(correlation(rank(high), rank(volume), 3)), 3)：最后对近3日内的相关系数分位值进行累加并取负号。
"""

# factor = (["-1*Sum(Rank(Corr(Rank($high),Rank($volumn),3)),3)"],
#           ["factor15"])

factor = (["$amount", "$change", "$close", "$high", "$low", "$open", "$volume", "$vwap"],
          ["$amount", "$change", "$close", "$high", "$low", "$open", "$volume", "$vwap"])
label = (["Ref($close, -2)/Ref($close, -1) - 1"], ["LABEL0"])

dataloader_config = {
    "class": "QlibDataLoader",
    "module_path": "qlib.data.dataset.loader",
    "kwargs": {
        "config": {
            "feature": factor,
            "label": label,
        },
        # "filter_pipe": [tt],
        "freq": 'day',
        "inst_processor": None,
        "swap_level": True,
    },
}

DEFAULT_LEARN_PROCESSORS = [
    {"class": "DropnaLabel"},
    {"class": "DropnaProcessor"},
    {"class": "CSZScoreNorm", "kwargs": {"fields_group": "label"}},
]
DEFAULT_INFER_PROCESSORS = [
    {"class": "ProcessInf", "kwargs": {}},
    {"class": "ZScoreNorm", "kwargs": {}},
    {"class": "Fillna", "kwargs": {}},
]
learn_proc = check_transform_proc(DEFAULT_LEARN_PROCESSORS,fit_start_time,fit_end_time)
infer_proc = check_transform_proc(DEFAULT_INFER_PROCESSORS,fit_start_time,fit_end_time)

# [{'class': 'DropnaLabel', 'kwargs': {}},
#  {'class': 'DropnaProcessor', 'kwargs': {}},
#  {'class': 'CSZScoreNorm', 'kwargs': {'fields_group': 'label'}}]
# [{'class': 'ProcessInf', 'kwargs': {}},
#  {'class': 'ZScoreNorm',
#   'kwargs': {'fit_start_time': '2020-01-01', 'fit_end_time': '2020-01-31'}},
#  {'class': 'Fillna', 'kwargs': {}}]

datahandler_config = {
    "class": "DataHandlerLP",
    "module_path": "qlib.data.dataset.handler",
    "kwargs":{
        "start_time":start_time,
        "end_time":end_time,
        "data_loader":dataloader_config,
        "learn_processors":DEFAULT_LEARN_PROCESSORS,
        "infer_processors":DEFAULT_INFER_PROCESSORS,
        "instruments":"test",

    }
}

dataset_config = {
    "class": "DatasetH",
    "module_path": "qlib.data.dataset",
    "kwargs":{
        "handler": datahandler_config,
        "segments": {
            "insample": (start_time, "2022-12-31"),
            "outsample": ("2023-01-01", end_time),
            },
    }

}

datahandler = init_instance_by_config(datahandler_config)
# dataset = init_instance_by_config(dataset_config)

# # features, example from SH0000001
# D.features(instruments=all_shsz, start_time='2020-01-01', end_time='2023-07-14', fields=['$close', '$factor', '$high']).head(50)


# mkt_csi300 = D.instruments(market='all')
# D.features(mkt_csi300, fields, start_time=start, end_time=end, freq='day')

# # dataLoader params
# features = ['$close', '$factor', '$high']
# features = ["adjclose", "amount", "change", "close", "factor", "high", "low", "open", "volume", "vwap", "daily_return"]


[29048:MainThread](2023-08-04 09:59:48,538) INFO - qlib.timer - [log.py:128] - Time cost: 11.865s | Loading data Done
[29048:MainThread](2023-08-04 09:59:51,659) INFO - qlib.timer - [log.py:128] - Time cost: 3.119s | ProcessInf Done
d:\anaconda3\envs\qlib\lib\site-packages\qlib\data\dataset\processor.py:256: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc(axis=1)[self.cols] = normalize(df[self.cols].values)
[29048:MainThread](2023-08-04 09:59:51,672) INFO - qlib.timer - [log.py:128] - Time cost: 0.011s | ZScoreNorm Done
[29048:MainThread](2023-08-04 09:59:51,675) INFO - qlib.timer - [log.py:128] - Time cost: 0.000s | Fillna Done
[29048:MainThread](2023-08-04 09:59:51,680) INFO - qlib.timer - [log.py:128] - Time cost: 0.003s | DropnaLabel Done
[29048:MainThread](2

In [17]:
datahandler.get_cols()

['$amount',
 '$change',
 '$close',
 '$high',
 '$low',
 '$open',
 '$volume',
 '$vwap',
 'LABEL0']

In [20]:
datahandler.fetch().head(30)

$amount   $change    $close     $high      $low  \
datetime   instrument                                                     
2023-01-03 SZ000001    2.000074  1.453624  0.064570  0.055258  0.010187   
           SZ002230   -0.078853  1.474880  1.024502  1.004423  0.959219   
           SZ002602   -0.792074 -0.821189 -1.304682 -1.306524 -1.307989   
2023-01-04 SZ000001    2.144464  1.208431  0.128774  0.121075  0.079356   
           SZ002230   -0.563502 -0.564072  1.008074  1.005906  1.036295   
           SZ002602   -0.755499  1.104862 -1.295798 -1.299614 -1.302147   
2023-01-05 SZ000001    1.425061  0.108096  0.147449  0.158022  0.167599   
           SZ002230   -0.521659 -0.016855  1.028196  1.004382  1.054563   
           SZ002602   -0.879905 -0.515394 -1.297048 -1.295215 -1.296956   
2023-01-06 SZ000001    0.730659  0.050546  0.163790  0.155712  0.180715   
           SZ002230   -0.533413 -0.852630  0.992363  0.995536  1.033980   
           SZ002602   -0.932020 -0.614981 -1.298957 -1.299608 -1.298259   
2023-01-09 SZ000001    0.537133  0.151649  0.184802  0.174187  0.185487   
           SZ002230   -0.648806 -0.206560  0.999860  0.996303  1.040876   
           SZ002602   -0.964257 -0.120563 -1.297706 -1.299623 -1.296982   
2023-01-10 SZ000001    0.837698 -1.249600  0.142778  0.175339  0.169982   
           SZ002230   -0.625755 -0.229363  1.005814  0.988893  1.025592   
           SZ002602   -0.826162  0.372566 -1.293297 -1.292758 -1.298298   
2023-01-11 SZ000001    0.180347  0.289955  0.169619  0.162632  0.169976   
           SZ002230   -0.337780 -1.350548  0.937071  0.986650  0.972150   
           SZ002602   -0.861437 -1.581229 -1.301523 -1.295259 -1.300234   
2023-01-12 SZ000001   -0.134077 -0.319252  0.169619  0.161478  0.186671   
           SZ002230   -0.673134 -0.192528  0.945284  0.934190  0.964517   
           SZ002602   -0.942021 -0.620414 -1.303432 -1.303409 -1.302831   
2023-01-13 SZ000001    0.377176  0.410764  0.202310  0.182269  0.189064   
           SZ002230   -0.772528 -0.376664  0.941591  0.920193  0.976768   
           SZ002602   -0.958951 -0.116884 -1.302116 -1.305237 -1.303427   
2023-01-16 SZ000001    1.361911  0.013336  0.217482  0.220369  0.224836   
           SZ002230    0.655972  1.888669  1.084977  1.090841  0.987427   
           SZ002602   -0.760365  0.284658 -1.298364 -1.298396 -1.303474   

                          $open   $volume     $vwap    LABEL0  
datetime   instrument                                          
2023-01-03 SZ000001    0.006654  2.297839  0.041744  0.168983  
           SZ002230    0.929092 -0.871556  1.000382  0.041196  
           SZ002602   -1.303632  0.819053 -1.306438 -0.462666  
2023-01-04 SZ000001    0.066563  2.289770  0.119417  0.110422  
           SZ002230    1.039588 -1.263026  1.011448 -0.807863  
           SZ002602   -1.302974  1.072113 -1.301132 -0.567874  
2023-01-05 SZ000001    0.147610  1.339650  0.155508  0.213312  
           SZ002230    1.030522 -1.232247  1.027005 -0.150621  
           SZ002602   -1.295950 -0.005028 -1.296605 -0.066123  
2023-01-06 SZ000001    0.159355  0.488442  0.167611 -1.211984  
           SZ002230    1.039562 -1.237553  1.003413 -0.174577  
           SZ002602   -1.295959 -0.420734 -1.299457  0.434409  
2023-01-09 SZ000001    0.188721  0.238186  0.184560  0.353820  
           SZ002230    1.014786 -1.331491  1.012741 -1.314982  
           SZ002602   -1.297249 -0.694602 -1.298294 -1.548693  
2023-01-10 SZ000001    0.189893  0.621974  0.163436 -0.265669  
           SZ002230    1.026790 -1.312865  1.011475 -0.136815  
           SZ002602   -1.297271  0.425389 -1.295049 -0.573483  
2023-01-11 SZ000001    0.153475 -0.173370  0.172323  0.477008  
           SZ002230    1.020750 -1.072064  0.968195 -0.323419  
           SZ002602   -1.290898  0.169169 -1.299523 -0.051691  
2023-01-12 SZ000001    0.191062 -0.544664  0.167117  0.072563  
           SZ002230    0.947840 -1.342151  0.945828  1.979576  
           SZ002602   -

In [2]:
# 从calendar中读取 指定的频率和时间范围，查看数据的时间范围
start,end = '2018-01-01','2023-02-28'
cal_d = D.calendar(start_time=start, end_time=end, freq='day')
print(cal_d[:2])

# 初始化一个stook pool 从instruments中，指定 market 为文件名，读取个股code
mkt_csi300 = D.instruments(market='csi300')   # {'market': 'all', 'filter_pipe': []}
mkt_csi300_list = D.list_instruments(instruments=mkt_csi300, start_time=start, end_time=end, as_list=True)

# 有filter api 用来过滤数据
# expressionDFilter = ExpressionDFilter(rule_expression='$close>2000')
# instruments = D.instruments(market='csi300', filter_pipe=[expressionDFilter])
# D.list_instruments(instruments=instruments, start_time='2020-01-01', end_time='2020-09-25', as_list=True)


[Timestamp('2018-01-02 00:00:00') Timestamp('2018-01-03 00:00:00')]


In [34]:
# 在这里做一些测试
import qlib
from qlib.utils import get_callable_kwargs
# from qlib.data.dataset import processor
# from qlib.data.dataset.processor import Processor
from qlib.contrib.data.handler import check_transform_proc

start_time = "2020-01-01"
end_time = "2023-05-26"
train_start_time = start_time
train_end_time = "2021-12-31"
val_start_time = "2022-01-01"
val_end_time = "2022-12-31"
test_start_time = "2023-01-01"
test_end_time = end_time

DEFAULT_LEARN_PROCESSORS = [
    {"class": "DropnaLabel"},
    {"class": "CSZScoreNorm", "kwargs": {"fields_group": "label"}},
]
DEFAULT_INFER_PROCESSORS = [
    {"class": "ProcessInf", "kwargs": {}},
    {"class": "ZScoreNorm", "kwargs": {}},
    {"class": "Fillna", "kwargs": {}},
]

a = {"class":"ZScoreNorm",
     "kwargs":{
         "fit_start_time": start_time,
         "fit_end_time": end_time,
     }
}
ll = [a]
check_transform_proc(DEFAULT_INFER_PROCESSORS,start_time,end_time)
# isinstance(a, Processor)
# a,b = get_callable_kwargs({"class": "ZScoreNorm", "kwargs": {"aaa":"bbb"}},processor)


[{'class': 'ProcessInf', 'kwargs': {}},
 {'class': 'ZScoreNorm',
  'kwargs': {'fit_start_time': '2020-01-01', 'fit_end_time': '2023-05-26'}},
 {'class': 'Fillna', 'kwargs': {}}]

In [27]:
a

qlib.data.dataset.processor.ZScoreNorm